In [ ]:
using Turing, Distributions
using PyPlot, PyCall

In [ ]:
y1 = 10*sin(0:0.2:2pi); y1 += randn(length(y1))
y2 = 10*sin(0:0.2:2pi); y2 += randn(length(y2))
Y = [y1'; y2']
N = size(Y)[end];  K = 5;

In [ ]:
@model FHMM(y) = begin
    s1 = tzeros(Int64, N)
    s2 = tzeros(Int64, N)
    m1 = tzeros(Real, K)
    m2 = tzeros(Real, K)
    T1 = Vector{Vector{Real}}(K)
    T2 = Vector{Vector{Real}}(K)
    for i = 1:K
        T1[i] ~ Dirichlet(ones(K)/K)
        T2[i] ~ Dirichlet(ones(K)/K)
        m1[i] ~ Normal(i, 0.01)
        m2[i] ~ Normal(i, 0.01)
    end
    s1[1] ~ Categorical(ones(Float64, K)/K)
    s2[1] ~ Categorical(ones(Float64, K)/K)
    for i = 2:N
        s1[i] ~ Categorical(vec(T1[s1[i-1]]))
        s2[i] ~ Categorical(vec(T2[s2[i-1]]))
        y[i] ~ MvNormal([m1[s1[i]], m2[s2[i]]], 0.01*ones(2))
    end
end

In [ ]:
g = Gibbs(300, HMC(1, 0.2, 5, :m1, :T1, :m2, :T2), PG(50, 1, :s1, :s2))
c = sample(FHMM(y), g);

In [ ]:
describe(c)

In [ ]:
m = c[:m][111];
s = c[:s][111];
PyPlot.plot(y, linestyle="None", marker="+", color = "r")
PyPlot.plot(m[s], linestyle="-", marker="*", color = "b")